# Optimisation des hyperparamètres

## Utilisation de keras tuner

In [1]:
import keras_tuner as kt
import keras

In [2]:
(X_train,y_train),(X_test,y_test)=keras.datasets.fashion_mnist.load_data()

Création de la fonction appelée par le tuner d'hyperparamètres

In [7]:
def build_model(hp):
    n_hidden = hp.Int("n_hidden",min_value=0,max_value=8,default=0)
    learning_rate = hp.Float("learning_rate",min_value=1e-4,max_value=1e-2,sampling="log") # par défaut, default = min_value # sampling log pour assurer une répartission égale entre 1e-4 et 1e-2
    n_neurons = hp.Int("n_neurons",min_value = 16,max_value = 256)
    optimizers = hp.Choice("optimizer",values = ["adam","sgd"])
    if optimizers == "sgd":
        optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    else:
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    
    model = keras.Sequential()
    model.add(keras.layers.Flatten())

    for _ in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons,activation="relu"))
    
    model.add(keras.layers.Dense(10,activation="softmax"))

    model.compile(optimizer=optimizer,
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    
    return model

Création d'un tuner qui cherche des combinaisons random d'hyperparamètres (qui viennent avant le fit du modele)

In [6]:
random_tuner = kt.RandomSearch(build_model, #va chercher parmi les models spécifié par cette fn
                               objective="val_accuracy", # va classer en fonction de l'accuracy du jeu de validation
                               max_trials=4, # ne va pas faire plus de 4 tirages parmi tous les choix de models possible
                               overwrite=True, # Réinitialise les données à chaque fois qu'on appelle seach()
                               directory="my_fashion_mnist", # données rangées dans le dossier my_fashion_mnist
                               project_name="my_rnd_research", # dans un sous-répertoire my_rns_research
                               seed=42)

# ATTENTION!!!
# Il vaut mieux en général mettre overwrite = False comme ça, on garde les données précédente et cela permet de comparer les précédents modèles
# aux nouveaux quand on va reappeler la fonction search()

In [7]:
random_tuner.search(X_train,y_train,epochs = 8, validation_split = 0.2)

Trial 4 Complete [00h 00m 39s]
val_accuracy: 0.6957499980926514

Best val_accuracy So Far: 0.6957499980926514
Total elapsed time: 00h 02m 12s


In [8]:
# Avoir le meilleur model parmi les 3 premiers
top3_models = random_tuner.get_best_models(num_models=3)
best_model = top3_models[0]


In [9]:
# Avoir les meilleurs params parmi les parmas des 3 models arrivés premiers
top3_params = random_tuner.get_best_hyperparameters(num_trials=3)
top3_params[0].values #top3_params[0] est un objet et on appelle ses valeurs

{'n_hidden': 8,
 'learning_rate': 0.00015159319577885927,
 'n_neurons': 248,
 'optimizer': 'adam'}

In [10]:
#Retourner les spécificités du meilleur model:
best_trial = random_tuner.oracle.get_best_trials(num_trials=1)[0]
best_trial.summary()

Trial 3 summary
Hyperparameters:
n_hidden: 8
learning_rate: 0.00015159319577885927
n_neurons: 248
optimizer: adam
Score: 0.6957499980926514


Régler les parmètres de fit du model (taille du lot, de quel manière preprocess les data ...)

In [8]:
class MyClassificationHypermodel(kt.HyperModel):

    def build(self, hp):
        return build_model(hp)
    
    def fit(self, hp, model, X,y, **kwargs):
        if hp.Boolean("normilize"):
            X = keras.layers.Normalization()(X)
        return model.fit(X,y,**kwargs) 


Trouver les hyper paramètres ainsi que s'il faut normaliser ou pas avec hyperBand

In [10]:
hyperband_tuner = kt.Hyperband(
    MyClassificationHypermodel(),
    objective="val_accuracy",
    factor=3, # commence à entrainer plein de modèles puis garde 1/factor des meilleurs et ainsi de suite jusqu'à ce qu'il n'en reste plus qu'un
    hyperband_iterations=2, #Réalise l'opération entière 2 fois
    max_epochs=8, 
    overwrite = True,
    directory = "my_fashion_mnist",
    project_name = "hyperband",
)

In [11]:
from pathlib import Path
root_logdir = Path(hyperband_tuner.project_dir) / "tensorboard"
tensorboard_cb = keras.callbacks.TensorBoard(root_logdir)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=2)
hyperband_tuner.search(X_train,y_train,epochs = 10,
                       validation_split = 0.2,
                       callbacks = [tensorboard_cb,early_stopping_cb])




Trial 20 Complete [00h 00m 51s]
val_accuracy: 0.8427500128746033

Best val_accuracy So Far: 0.8575000166893005
Total elapsed time: 00h 12m 01s


Une meillure solution est d'utiliser le BayesianOptimiser qui cherche la meilleure combianaiseon possible de params, mais pas de façon aléatoire. Il privilégie les hyperparamètres qui semblent les plus prometteurs au cours de l'exploration

alpha est le niveau de bruit que l'on souhaite dans la mesure des preformances entre essai

beta est l'importance de l'exploration du tuner vis à vis de l'xploitation des bonnes régions connues de l'epace des hyperpaprams

In [ ]:
bayesian_tuner = kt.BayesianOptimization(
    MyClassificationHypermodel(),
    objective='val_accuracy',
    seed = 42,
    max_trials = 10, # nombre de fois qu'il essai un nouveau model
    alpha = 1e-4,
    beta=2.6,
    overwrite = True,
    directory = "my_fashion_mnist",
    project_name = "bayesian"
)

# bayesian_tuner.search([...])